# Execution of the Bell circuit

The usual first experiment in Quantum Computing is to simulate a 2-qubit Bell state.

## Creating the EPR/Bell state 

Let us create first the related circuit.

In [1]:
from mpqp import QCircuit
from mpqp.gates import H, CNOT

In [2]:
circuit = QCircuit([H(0), CNOT(0,1)], label="Bell state")
print(circuit)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


## Run the circuit on a local device

We can execute a circuit on a local simulator by calling the function ``run`` and precising the device.

In [3]:
from mpqp.execution import run, ATOSDevice

When no measure is added to the circuit, running the circuit will consist in extracting the state-vector at the output of the circuit.

In [4]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.amplitudes)
print(result.probabilities)

Result: ATOSDevice, MYQLM_PYLINALG

        State vector: [0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
        Probabilities: [0.5 0.  0.  0.5]
        Number of qubits: 2


[0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
[0.5 0.  0.  0.5]


We can also add to the circuit a ``BasisMeasure``, consisting in sample the state in a given basis. By default, the basis is the computational one.

In [5]:
from mpqp.measures import BasisMeasure

We precise which qubits we can to measure by inputting a list of indices, and precising the number of shots. When ``shots=0``, we end up in the same case as before, a statevector simulation.

In [6]:
circuit.add(BasisMeasure([0,1], shots=0))
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)

Result: ATOSDevice, MYQLM_PYLINALG

        State vector: [0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
        Probabilities: [0.5 0.  0.  0.5]
        Number of qubits: 2




When we precise a number of shots, the circuit will be sampled and a list of ``Sample`` will be returned, precising the counts for each state of the basis.

In [7]:
circuit = circuit.without_measurements()
circuit.add(BasisMeasure([0,1], shots=1024))

In [8]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.samples)
print(result.counts)
print(result.probabilities)

Result: ATOSDevice, MYQLM_PYLINALG
Counts: [493, 0, 0, 531]
Probabilities: [0.48144531 0.         0.         0.51855469]
State: 00, Index: 0, Count: 493, Probability: 0.4814453125
State: 11, Index: 3, Count: 531, Probability: 0.5185546875
Error: 0.015621867383234688


[State: 00, Index: 0, Count: 493, Probability: 0.4814453125, State: 11, Index: 3, Count: 531, Probability: 0.5185546875]
[493, 0, 0, 531]
[0.48144531 0.         0.         0.51855469]


## Run the circuit on multiple devices

By using the same function ``run`` we can execute the circuit on several simulators at the time. One just has to give a list of devices instead of a single device.

In [9]:
from mpqp.execution import IBMDevice, AWSDevice

In [10]:
results = run(circuit, [ATOSDevice.MYQLM_PYLINALG, IBMDevice.AER_SIMULATOR, AWSDevice.BRAKET_LOCAL_SIMULATOR])
print(results)
print('---------')
print(results[0])

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.


BatchResult: 3 results
Result: ATOSDevice, MYQLM_PYLINALG
Counts: [520, 0, 0, 504]
Probabilities: [0.5078125 0.        0.        0.4921875]
State: 11, Index: 3, Count: 504, Probability: 0.4921875
State: 00, Index: 0, Count: 520, Probability: 0.5078125
Error: 0.01563072658989357

Result: IBMDevice, AER_SIMULATOR
Counts: [515, 0, 0, 509]
Probabilities: [0.50292969 0.         0.         0.49707031]
State: 00, Index: 0, Count: 515, Probability: 0.5029296875
State: 11, Index: 3, Count: 509, Probability: 0.4970703125
Error: None

Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
Counts: [513, 0, 0, 511]
Probabilities: [0.50097656 0.         0.         0.49902344]
State: 00, Index: 0, Count: 513, Probability: 0.5009765625
State: 11, Index: 3, Count: 511, Probability: 0.4990234375
Error: None


---------
Result: ATOSDevice, MYQLM_PYLINALG
Counts: [520, 0, 0, 504]
Probabilities: [0.5078125 0.        0.        0.4921875]
State: 11, Index: 3, Count: 504, Probability: 0.4921875
State: 00, Index: 0, Count:

## Run or submit the circuit on a remote device

To execute the circuit on remote device, one can use the exact same process as with local devices. A call of the function ``run``on a remote device will make the program wait until the computation ends on the provider's side. One or several devices can still be given in parameter.

In [11]:
result = run(circuit, IBMDevice.IBMQ_QASM_SIMULATOR)
print(result)

Result: IBMDevice, IBMQ_QASM_SIMULATOR
Counts: [497, 0, 0, 527]
Probabilities: [0.48535156 0.         0.         0.51464844]
State: 00, Index: 0, Count: 497, Probability: 0.4853515625
State: 11, Index: 3, Count: 527, Probability: 0.5146484375
Error: None




However, it is also possible to submit the job in an asynchronous way using the function ``submit``.

In [12]:
from mpqp.execution import submit

By submitting the circuit to a remote device, we retrieve the id of the job on the provider side, as well as the MPQP job related with this submission, which contains additional information, such as the ``status`` of the job.

In [13]:
job_id, job = submit(circuit, IBMDevice.IBMQ_QASM_SIMULATOR)
print(job_id)

cn9e9gsd06qe0i2tj4hg


Once the computation is done, we use the function ``remote_result_from_id`` for retrieving the result. If the job is not completed, the function will wait (blocking) until it is done.

In [14]:
from mpqp.execution import remote_result_from_id

In [15]:
result = remote_result_from_id(job_id, IBMDevice.IBMQ_QASM_SIMULATOR)
print(result)

Result: IBMDevice, IBMQ_QASM_SIMULATOR
Counts: [505, 0, 0, 519]
Probabilities: [0.49316406 0.         0.         0.50683594]
State: 00, Index: 0, Count: 505, Probability: 0.4931640625
State: 11, Index: 3, Count: 519, Probability: 0.5068359375
Error: None


